<a href="https://colab.research.google.com/github/BrunoMog/PROJETOS-IF687/blob/main/mlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IF867 - Introdução à Aprendizagem Profunda

### 1ª atividade prática

Discente(s): Bruno Antonio dos Santos Bezerra

Período: 7°

### Instruções e Requisitos
- Objetivo: Implementar e treinar um Multilayer Perceptron (MLP), inteiramente em [NumPy](https://numpy.org/doc/stable/) ou [Numba](https://numba.readthedocs.io/en/stable/index.html), sem o uso de bibliotecas de aprendizado profundo.
- A atividade pode ser feita em dupla.

### Tarefas

__Implementação (50%):__

- Construa um MLP com uma camada de entrada, pelo menos duas camadas ocultas e uma camada de saída.
- Implemente pelo menos duas funções de ativação diferentes para as camadas ocultas; use Sigmoid e Linear para a camada de saída.
- Implemente forward e backpropagation.
- Implemente um otimizador de sua escolha, adequado ao problema abordado.
- Implemente as funções de treinamento e avaliação.

__Aplicação (30%):__

  Teste se os seus modelos estão funcionando bem com as seguintes tarefas:
  - Regressão
  - Classificação binária

__Experimentação (20%):__

  Teste os seus modelos com variações na arquitetura, no pré-processamento, etc. Escolha pelo menos uma das seguintes opções:
  - Variações na inicialização de pesos
  - Variações na arquitetura
  - Implementação de técnicas de regularização
  - Visualização das ativações e gradientes

***Bônus:*** Implemente o MLP utilizando uma biblioteca de machine learning (ex.: [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/?hl=pt-br), [tinygrad](https://docs.tinygrad.org/), [Jax](https://jax.readthedocs.io/en/latest/quickstart.html)) e teste-o em uma das aplicações e em um dos experimentos propostos. O bônus pode substituir um dos desafios de aplicação ou experimentos feitos em NumPy, ou simplesmente somar pontos para a pontuação geral.

### Datasets recomendados:
Aqui estão alguns datasets recomendados, mas fica a cargo do aluno escolher os datasets que utilizará na atividade, podendo escolher um dataset não listado abaixo.
- Classificação

  - [Iris](https://archive.ics.uci.edu/dataset/53/iris)
  - [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)
  - [CDC Diabetes Health Indicators](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators)

- Regressão

  - [Air Quality](https://archive.ics.uci.edu/dataset/360/air+quality)
  - [Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance)
  - [Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)

### Requisitos para Entrega

Um notebook Jupyter (de preferência, o link do colab) ou script Python contendo:

- Código: Implementação completa da MLP.
- Gráficos e Análises: Gráficos da curva de perda, ativações, gradientes e insights do treinamento, resultantes dos experimentos com parada antecipada e diferentes técnicas de regularização.
- Relatório: Um breve relatório detalhando o impacto de várias configurações de hiperparâmetros(ex.: inicialização de pesos, número de camadas ocultas e neurônios) e métodos de regularização no desempenho do modelo.


## Início da implementação


In [34]:
#importanto biblioteca
import numpy as np

In [35]:
#setando parâmetros da rede mlp

input_size = 4
hidden_size = 5
output_size = 3

#cada camada intermediária pode ter o mesmo número de unidades computacionais para fins de simplificação ou apenas terem valores diferentes, nessa implementação vamos adotar a segunda abordagem
#para cada camada intermediária - hidden layer, terá um valor relacionado a quantidade de unidades computacionais

hidden_layer_size = [5, 5, 5, 5, 5]

# serão implementadas três funções de ativação para as camadas intermediárias a relu, sigmoid e linear pela qual será indicada, e para a camdada de saída apenas sigmoid ou linear
# a primeira função será para camadas intermediárias e a segunda para a camada de saída

activation_functions = ['sigmoid', 'linear']

# o número máximo de épocas que o modelo será treinado
# o objetivo é que não precise chegar ao número máximo de épocas, quando o modelo apresentar overfitting o treinamento já acabe

MAX_EPOCH = 100000

# taxa de aprendizado que será utilizada

learning_rate = 0.03

# tamanho dos mini-batchs que será utilizado dos dados de treinamento e teste

batch_size_train = 100

batch_size_test = 20

In [40]:
#setando o gerador de números aleatórios para setar os pesos iniciais

weigths_seed = 115
rng = np.random.default_rng(weigths_seed)

# como a rede mlp é totalmente conectada, o número de pesos gerados entre camadas segirá sempre o padrão layer[i]*layer[i+1]

weights = []

for i in range(hidden_size+1):

  if i == 0:

    weights.append(rng.random(size = (input_size, hidden_layer_size[i], 1)))

  elif i < hidden_size:

    weights.append(rng.random((hidden_layer_size[i-1], hidden_layer_size[i], 1)))

  else:

    weights.append(rng.random((hidden_layer_size[i-1], output_size, 1)))



In [63]:
# primeiro index é para referir a qual camada, sempre haverá hidden_layers+1 camada de pesos
# o segundo index para referenciar qual unidade computacional da camada
# o terceiro index para referenciar o peso a qual unidade computacional da próxima camada
# o quarto index para obter o valor
print(weights[5][0][0][0])

0.9024653423513748


### Funções para o treinamento da rede MLP

#### fase foward e backward


In [ ]:
# funções de ativação

# função relu normal
def function_Relu(x):

  return max(0, x)

# função sigmoid normal
def function_Sigmoid(x):

  return 1/(1+np.exp(-x))

# função linear entre 0 e 1
def function_Linear(x):

  return max(0, min(1, x))

In [ ]:
# processo de foward da rede

def foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, data):

  # o resultado de cada valor multiplicado pelo seu peso será armazenado no valor parcial

  resultado_parcial = [[[] for j in range(hidden_layer_size[i])] for i in range(hidden_size)]

  resultado_parcial.append([[]for i in range(output_size)])

  # calcular o resultado camada por camada

  for i in range(hidden_size+1):

    # no primeiro caso será a camada de dados

    if i == 0:

      input = data

    # o caso em que estamos calculando o resuldado das camadas intermediárias

    if(i < hidden_size):

      for j in range(len(input)):

        for k in range(hidden_layer_size[i]):

          resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])


      aux = np.zeros(hidden_layer_size[i])

      for j in range(hidden_layer_size[i]):

        aux[j] = np.sum(resultado_parcial[i][j])

      # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

      if activation_functions[0] == 'relu':

        for j in range(len(aux)):

          aux[j] = function_Relu(aux[j])

      elif activation_functions[0] == 'sigmoid':

        for j in range(len(aux)):

          aux[j] = function_Sigmoid(aux[j])

      elif activation_functions[0] == 'linear':

        for j in range(len(aux)):

          aux[j] = function_Linear(aux[j])


    # para o caso contrário

    else:

      for j in range(len(input)):

        for k in range(output_size):

          resultado_parcial[i][k].append(weights[i][j][k][0]*input[j])

      aux = np.zeros(output_size)

      for j in range(output_size):

        aux[j] = np.sum(resultado_parcial[i][j])

      # após ter os resultados parciais da soma ponderada das entradas daquela camada, esse valor irá passar pela função de ativação

      if activation_functions[0] == 'relu':

        for j in range(len(aux)):

          aux[j] = function_Relu(aux[j])

      elif activation_functions[0] == 'sigmoid':

        for j in range(len(aux)):

          aux[j] = function_Sigmoid(aux[j])

      elif activation_functions[0] == 'linear':

        for j in range(len(aux)):

          aux[j] = function_Linear(aux[j])

    # após ter o resultado parcial da camada, o input da próxima iteração será o resultado dessa

    input = aux

  return input


In [ ]:
# o treinamento fará no máximo MAX_EPOCH iterações
# ele irá parar quando a acurácia dos testes diminuir em 5 épocas seguidas
# após o processo de treinamento será retornado os pesos ótimos para o problema encontrado

def treinamento(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, MAX_EPOCH, learning_rate, batch_size_train, batch_size_test, weights, data_train, data_test, data_train_label, data_test_label):

  for epoch in range(MAX_EPOCH):

    # atualizando os batchs de treino e de teste

    batch_index = rng.integers(len(data_train), size = batch_size_train)

    batch_train = data_train[batch_index]

    batch_train_label = data_train_label[batch_index]

    batch_index = rng.integers(len(data_test), size = batch_size_test)

    batch_test = data_test[batch_index]

    batch_test_label = data_test_label[batch_index]

    # atualizando os pesos de acordo com o erro no batch do treino
    # a função de custo será a MSE

    for i in range(batch_size_train):

      #resultado do processo de foward

      foward_result = foward(input_size, hidden_size, output_size, hidden_layer_size, activation_functions, weights, batch_train[i])





  return weights